# Satifsying requirements

In [ ]:
#!pip install pennylane --upgrade

In [ ]:
#!pip install torch

In [ ]:
import torch
import pennylane as qml
import numpy as np

%matplotlib inline

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

# Downloading data

In [ ]:
from torchvision import datasets
from torchvision.transforms import ToTensor

train_data = datasets.MNIST(
    root = 'data',
    train = True,                         
    transform = ToTensor(), 
    download = True,            
)
test_data = datasets.MNIST(
    root = 'data', 
    train = False, 
    transform = ToTensor()
)

# Preparing data with DataLoaders

In [ ]:
from torch.utils.data import DataLoader

loaders = {
    'train' : torch.utils.data.DataLoader(train_data, 
                                          batch_size=100, 
                                          shuffle=True, 
                                          num_workers=1),
    
    'test'  : torch.utils.data.DataLoader(test_data, 
                                          batch_size=100, 
                                          shuffle=True, 
                                          num_workers=1),
}

# Defining a NN

In [ ]:
n_qubits = 10
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def qnode(inputs, weights):
    qml.templates.AngleEmbedding(inputs, wires=range(n_qubits))
    qml.templates.BasicEntanglerLayers(weights, wires=range(n_qubits))
    return [qml.expval(qml.PauliZ(wires=i)) for i in range(n_qubits)]

In [ ]:
n_layers = 5
weight_shapes = {"weights": (n_layers, n_qubits)}
qlayer = qml.qnn.TorchLayer(qnode, weight_shapes)

In [ ]:
import torch.nn as nn

class HybridNN(nn.Module):
    def __init__(self):
        super(HybridNN, self).__init__()
        self.conv1 = nn.Sequential(         
            nn.Conv2d(
                in_channels=1,              
                out_channels=16,            
                kernel_size=5,              
                stride=1,                   
                padding=2,                  
            ),                              
            nn.ReLU(),                      
            nn.MaxPool2d(kernel_size=2),    
        )
        self.conv2 = nn.Sequential(         
            nn.Conv2d(
                in_channels=16,              
                out_channels=32,            
                kernel_size=5,              
                stride=1,                   
                padding=2,    
            ),     
            nn.ReLU(),                      
            nn.MaxPool2d(kernel_size=2),                
        )
        self.fc_1 = nn.Linear(32 * 7 * 7, 50)
        
        # LIST USAGE?
        self.qlayer_1 = qml.qnn.TorchLayer(qnode, weight_shapes)
        self.qlayer_2 = qml.qnn.TorchLayer(qnode, weight_shapes)
        self.qlayer_3 = qml.qnn.TorchLayer(qnode, weight_shapes)
        self.qlayer_4 = qml.qnn.TorchLayer(qnode, weight_shapes)
        self.qlayer_5 = qml.qnn.TorchLayer(qnode, weight_shapes)
        
        self.fc_2 = nn.Linear(50, 10)
        self.out = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        # flatten the output of conv2 to (batch_size, 32 * 7 * 7)
        x = x.view(x.size(0), -1) 
        x = self.fc_1(x)
        
        x_1, x_2, x_3, x_4, x_5 = torch.split(x, 10, dim=1) # second argument is number of elements in one new tensor
        
        x_1 = self.qlayer_1(x_1)
        x_2 = self.qlayer_2(x_2)
        x_3 = self.qlayer_3(x_3)
        x_4 = self.qlayer_4(x_4)
        x_5 = self.qlayer_5(x_5)
        
        x = torch.cat([x_1, x_2, x_3, x_4, x_5], axis=1)
        
        x = self.fc_2(x)
        
        #logits = self.out(x)
        logits = x
        return logits

In [ ]:
hnn = HybridNN().to(device)
print(hnn)

# Training

In [ ]:
loss_func = nn.CrossEntropyLoss()

In [ ]:
from torch import optim

optimizer = optim.Adam(hnn.parameters(), lr = 0.01)  

In [ ]:
from tqdm.notebook import trange
from torch.autograd import Variable

def train(num_epochs, model, loaders):
    
    model.train()
        
    # Train the model
    total_step = len(loaders['train'])
        
    for epoch in trange(num_epochs):
        for i, (images, labels) in enumerate(loaders['train']):
            
            b_x, b_y = images.to(device), labels.to(device)

            #b_x = Variable(images)   # batch x
            #b_y = Variable(labels)   # batch y

            output = model(b_x)             
            loss = loss_func(output, b_y)
            
            # clear gradients for this training step   
            optimizer.zero_grad()           
            
            # backpropagation, compute gradients 
            loss.backward()    
            # apply gradients             
            optimizer.step()                
            
            if (i+1) % 100 == 0:
                print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                       .format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))
        print('\n')

In [ ]:
num_epochs = 10

train(num_epochs, hnn, loaders)